OPEN AI_CLIP GIT
https://github.com/openai/CLIP/blob/main/notebooks/Interacting_with_CLIP.ipynb

In [3]:
%pip install ftfy regex tqdm
%pip install git+https://github.com/openai/CLIP.git

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Cloning https://github.com/openai/CLIP.git to c:\users\user\appdata\local\temp\pip-req-build-qo78tci9
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\user\AppData\Local\Temp\pip-req-build-qo78tci9'
You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import os
import clip
import torch
from torch import nn, optim

from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder

%matplotlib inline
BATCH_SIZE = 32
EPOCH = 5

device = print("cuda" if torch.cuda.is_available() else "cpu")

cpu


# Prepare the Model and Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
model = model.to(torch.float32)

100%|███████████████████████████████████████| 338M/338M [00:13<00:00, 26.3MiB/s]


In [6]:
data_folder = '/content/drive/MyDrive/Study/ML_TeamProject/newData_384'

class_folders = os.listdir(data_folder)
image_paths= []  # List to store image paths and labels as tuples

for class_idx, class_folder in enumerate(class_folders):
    class_folder_path = os.path.join(data_folder, class_folder)
    image_files = os.listdir(class_folder_path)

    for image_file in image_files:
        image_path = os.path.join(class_folder_path, image_file)
        image_paths.append((image_path, class_idx, class_folder))

# (image path, index, label) tuples

print(image_paths[1])

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '/content/drive/MyDrive/Study/ML_TeamProject/newData_384'

In [6]:
class MyDataset(Dataset):
    def __init__(self, data, preprocess):
        self.preprocess = preprocess
        self.image_paths = []
        self.labels = []
        self.label_idx = []

        for img_path, index ,label in data:
            self.image_paths.append(img_path)
            self.label_idx.append(index)
            self.labels.append(label)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        image = self.preprocess(image)
        label = self.labels[idx]
        label_idx = self.label_idx[idx]
        return image, label_idx, label

# 데이터 전처리 및 데이터 로더 생성
dataset = MyDataset(image_paths, preprocess)

# 데이터 분할 (train, validation, test)
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# 데이터 로더 생성
trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

len(train_dataset), len(val_dataset)


(2185, 624)

In [7]:
from numpy.lib import shape_base
for batch in trainloader:
    images, _, labels = batch
    print("Image Path:", images[0].shape)
    print("Label:", labels[0])
    print("To Token:", clip.tokenize(labels[0]))
    break

Image Path: torch.Size([3, 224, 224])
Label: the_blusher
To Token: tensor([[49406,   518,   318,  1263,  4510, 49407,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], dtype=torch.int32)


# Building Features

In [8]:
# image_inputs = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
# text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in dataset.classes]).to(device)

# image_inputs.shape, text_inputs.shape

# Training

In [8]:
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(trainloader)*EPOCH)

In [10]:
from tqdm.notebook import tqdm

if device == "cpu":
  model.float()
else :
  clip.model.convert_weights(model)

for epoch in range(EPOCH):
    print(f"Epoch: {epoch+1}")
    pbar = tqdm(trainloader, total=len(trainloader))
    for batch in pbar:
        optimizer.zero_grad()

        images, _, label = batch
        images= images.to(device)
        labels = clip.tokenize(label).to(device)

        # Forward pass
        logits_per_image, logits_per_text = model(images, labels)

        # Compute loss
        # ground_truth = torch.arange(BATCH_SIZE).to(device)
        ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2

        # Backward pass
        total_loss.backward()
        if device == "cpu":
            optimizer.step()
        else :
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)

        pbar.set_description(f"Epoch {epoch}/{EPOCH}, Loss: {total_loss.item():.4f}")


Epoch: 1


  0%|          | 0/69 [00:00<?, ?it/s]

RuntimeError: ignored